In [4]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth

# Đọc ảnh
img1 = cv2.imread('2.jpg', 0)  # Template Image (ảnh mẫu)
img2 = cv2.imread('1.jpg')     # Search Image (ảnh cần tìm kiếm)
img_rgb = cv2.imread('1.jpg')  # Ảnh để vẽ kết quả


In [5]:
# Tạo SIFT detector
alg = cv2.SIFT_create()

# Tìm keypoints và descriptors
kp1, des1 = alg.detectAndCompute(img1, None)
kp2, des2 = alg.detectAndCompute(img2, None)

print(f"Số keypoints trong ảnh mẫu: {len(kp1)}")
print(f"Số keypoints trong ảnh tìm kiếm: {len(kp2)}")


error: OpenCV(4.6.0) C:\b\abs_f8n1j3l9l0\croot\opencv-suite_1691622637237\work\modules\features2d\src\sift.dispatch.cpp:477: error: (-5:Bad argument) image is empty or has incorrect depth (!=CV_8U) in function 'cv::SIFT_Impl::detectAndCompute'


In [ ]:
# Chuyển keypoints thành mảng numpy
x = np.array([kp.pt for kp in kp2])

# Ước lượng bandwidth cho MeanShift
bandwidth = estimate_bandwidth(x, quantile=0.1, n_samples=500)

# Áp dụng MeanShift clustering
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, cluster_all=True)
ms.fit(x)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

n_clusters_ = len(np.unique(labels))
print("Số cụm được tìm thấy:", n_clusters_)

# Gom nhóm keypoints theo từng cụm
s = [[] for _ in range(n_clusters_)]
for i, label in enumerate(labels):
    s[label].append(kp2[i])


In [ ]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)

des1 = np.float32(des1)
des2_ = np.float32(des2)

for i in range(n_clusters_):
    kp2 = s[i]

    if len(kp2) < 2 or len(kp1) < 2:
        continue

    # Lấy descriptors tương ứng với keypoints trong cụm
    d_indices = [kp2.index(kp) for kp in kp2]
    des2 = des2_[d_indices, :]

    matches = flann.knnMatch(des1, des2, k=2)

    # Áp dụng Lowe’s ratio test
    good = [m for m, n in matches if m.distance < 0.7 * n.distance]

    if len(good) > 3:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 2)

        if M is not None:
            h, w = img1.shape
            corners = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
            transformedCorners = cv2.perspectiveTransform(corners, M)

            x, y = int(transformedCorners[0][0][0]), int(transformedCorners[0][0][1])

            # Vẽ hình chữ nhật quanh khu vực tìm thấy
            cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (0, 0, 255), 3)
            img2 = cv2.polylines(img2, [np.int32(transformedCorners)], True, (0, 0, 255), 2, cv2.LINE_AA)


In [ ]:
plt.figure(figsize=(10,5))

# Hiển thị ảnh đã tìm thấy đối tượng
plt.subplot(1,2,1)
plt.imshow(cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB))
plt.title("Detected Objects")

# Hiển thị ảnh với đường viền tìm thấy
plt.subplot(1,2,2)
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
plt.title("Matched Keypoints")

plt.show()

# Lưu ảnh kết quả
cv2.imwrite('result.jpg', img_rgb)
